In [2]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None

In [3]:
with open('match.json') as file:
    data = json.load(file)

In [4]:
data['events'][0].keys()

dict_keys(['id', 'matchId', 'matchPeriod', 'minute', 'second', 'matchTimestamp', 'videoTimestamp', 'relatedEventId', 'type', 'location', 'team', 'opponentTeam', 'player', 'pass', 'shot', 'groundDuel', 'aerialDuel', 'infraction', 'carry', 'possession'])

In [5]:
# omzetten in dataframe
df = pd.DataFrame.from_dict(data['events'])

# df.head()

In [6]:
# type kolom splitsen in twee nieuwe
df[['primary', 'secondary']] = df['type'].apply(pd.Series)

# df.head(3)

In [7]:
# player kolom opsplitsen
player_detail = df['player'].apply(pd.Series)
df[['playerId', 'playerName', 'playerPosition']] = player_detail

# df.head(3)

In [ ]:
# PASSES APART ZETTEN
# verzamel passes in nieuwe df
passes_df = df[df['primary'] == 'pass'].copy()
# passes_df['pass'][0].keys()

# pass kolom opsplitsen
pass_detail = passes_df['pass'].apply(pd.Series)
passes_df = pd.concat([passes_df, pass_detail], axis = 1)

# locations opsplitsen
passes_df[['originX', 'originY']] = passes_df['location'].apply(pd.Series)
passes_df[['endX', 'endY']] = passes_df['endLocation'].apply(pd.Series)

# teams opsplitsen
passes_df[['teamId', 'teamName', 'teamFormation']] = passes_df['team'].apply(pd.Series)
passes_df[['oppositionId', 'oppositionName', 'oppositionFormation']] = passes_df['opponentTeam'].apply(pd.Series)

# receiving player opsplitsen
passes_df[['recPlayerId', 'recPlayerName', 'recPlayerPosition']] = passes_df['recipient'].apply(pd.Series)

# passes_df.head()

In [9]:
# Overzicht van alle secondaries (kolom van lists) maken
secondaries = []

for record in passes_df['secondary']:
    for type in record:
        secondaries.append(type)

set(secondaries)

# Alle pass to penalty area verzamelen
penalty_entries_df = passes_df[passes_df['secondary'].apply(lambda x: 'pass_to_penalty_area' in x)]
# penalty_entries_df.head(3)

In [39]:
# WELKE SPELERS GEVEN MEESTE PASSES
# groeperen volgens naam en of pass aankomt
passes_by_player = passes_df.groupby(['playerName', 'accurate'])

# count geven van (niet) aangekomen passes per speler
passes_by_player = passes_by_player.count()['id'].reset_index().pivot(index = 'playerName', columns = 'accurate')

# NaN vervangen door 0
passes_by_player = passes_by_player.fillna(0)

# kolommen opkuisen
passes_by_player.columns = passes_by_player.columns.droplevel()
passes_by_player.columns = ['incomplete', 'complete']

# analyse
passes_by_player['total'] = passes_by_player['incomplete'] + passes_by_player['complete']
passes_by_player['percentage'] = ((passes_by_player['complete'] / passes_by_player['total']) * 100).round(1)
passes_by_player.reset_index(level = 0, inplace = True)

# sorteren
passes_by_player = passes_by_player.sort_values('percentage', ascending = False)
passes_by_player

,playerName,incomplete,complete,total,percentage
10,J. Grealish,0.0,19.0,19.0,100.0
4,C. Jones,2.0,40.0,42.0,95.2
2,Alisson,2.0,37.0,39.0,94.9
0,A. Laporte,3.0,51.0,54.0,94.4
21,Rodri,3.0,45.0,48.0,93.8
22,Rúben Dias,4.0,49.0,53.0,92.5
24,V. van Dijk,6.0,72.0,78.0,92.3
12,J. Matip,6.0,66.0,72.0,91.7
3,Bernardo Silva,5.0,45.0,50.0,90.0
7,Fabinho,4.0,27.0,31.0,87.1


In [38]:
# SHOTS DATAFRAME

# nieuwe dataframe maken voor shots
shots_df = df[df['primary'] == 'shot'].copy()
# shots_df['shot'][173].keys()

# kolommen opsplitsen
shot_detail = shots_df['shot'].apply(pd.Series)
shots_df = pd.concat([shots_df, shot_detail], axis = 1)

# locations opsplitsen
shots_df[['originX', 'originY']] = shots_df['location'].apply(pd.Series)

# teams opsplitsen
shots_df[['teamId', 'teamName', 'teamFormation']] = shots_df['team'].apply(pd.Series)
shots_df[['oppositionId', 'oppositionName', 'oppositionFormation']] = shots_df['opponentTeam'].apply(pd.Series)

shots_df.head(4)

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,type,location,team,opponentTeam,player,pass,shot,groundDuel,aerialDuel,infraction,carry,possession,primary,secondary,playerId,playerName,playerPosition,bodyPart,isGoal,onTarget,goalZone,xg,postShotXg,goalkeeperActionId,goalkeeper,originX,originY,teamId,teamName,teamFormation,oppositionId,oppositionName,oppositionFormation
173,1074869036,5234875,1H,7,14,00:07:14.382,440.382856,NaN,"{'primary': 'shot', 'secondary': ['shot_after_...","{'x': 87, 'y': 32}","{'id': 1612, 'name': 'Liverpool', 'formation':...","{'id': 1625, 'name': 'Manchester City', 'forma...","{'id': 120353, 'name': 'Mohamed Salah', 'posit...",None,"{'bodyPart': 'left_foot', 'isGoal': False, 'on...",None,None,None,None,"{'id': 1074869028, 'duration': '2.819421', 'ty...",shot,"[shot_after_corner, touch_in_box]",120353.0,Mohamed Salah,RWF,left_foot,False,False,bc,0.01198,NaN,NaN,None,87,32,1612,Liverpool,4-3-3,1625,Manchester City,4-3-3
348,1074869450,5234875,1H,14,39,00:14:39.525,885.525798,NaN,"{'primary': 'shot', 'secondary': ['opportunity...","{'x': 90, 'y': 30}","{'id': 1625, 'name': 'Manchester City', 'forma...","{'id': 1612, 'name': 'Liverpool', 'formation':...","{'id': 11051, 'name': 'J. Grealish', 'position...",None,"{'bodyPart': 'right_foot', 'isGoal': False, 'o...",None,None,None,None,"{'id': 1074869418, 'duration': '56.6840225', '...",shot,"[opportunity, touch_in_box]",11051.0,J. Grealish,CF,right_foot,False,False,bc,0.05322,NaN,NaN,None,90,30,1625,Manchester City,4-3-3,1612,Liverpool,4-3-3
477,1074869693,5234875,1H,20,14,00:20:14.579,1220.579046,NaN,"{'primary': 'shot', 'secondary': ['opportunity...","{'x': 94, 'y': 37}","{'id': 1625, 'name': 'Manchester City', 'forma...","{'id': 1612, 'name': 'Liverpool', 'formation':...","{'id': 447205, 'name': 'P. Foden', 'position':...",None,"{'bodyPart': 'left_foot', 'isGoal': False, 'on...",None,None,None,None,"{'id': 1074869677, 'duration': '8.471544', 'ty...",shot,"[opportunity, touch_in_box]",447205.0,P. Foden,LWF,left_foot,False,True,glb,0.18540,0.4031,1.074869e+09,"{'id': 214220, 'name': 'Alisson'}",94,37,1625,Manchester City,4-3-3,1612,Liverpool,4-3-3
542,1074869799,5234875,1H,23,39,00:23:39.028,1425.028313,NaN,"{'primary': 'shot', 'secondary': ['opportunity...","{'x': 92, 'y': 33}","{'id': 1625, 'name': 'Manchester City', 'forma...","{'id': 1612, 'name': 'Liverpool', 'formation':...","{'id': 38021, 'name': 'K. De Bruyne', 'positio...",None,"{'bodyPart': 'left_foot', 'isGoal': False, 'on...",None,None,None,None,"{'id': 1074869781, 'duration': '13.6972145', '...",shot,"[opportunity, touch_in_box]",38021.0,K. De Bruyne,RCMF3,left_foot,False,False,obr,0.07633,NaN,NaN,None,92,33,1625,Manchester City,4-3-3,1612,Liverpool,4-3-3
